In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
# Libraries
import pandas as pd
import glob
import numpy as np

In [30]:
# Annotations file
data_path = "/content/drive/MyDrive/Hackathon_HiGency/data/dataset_set"
df_walid = pd.read_pickle("/content/drive/MyDrive/Hackathon_HiGency/data/test_car_annotations_fixed.pkl")
carbon_pred_np = np.load("/content/drive/MyDrive/Hackathon_HiGency/data/pred_carbon.npy")
carbon_pred_df = pd.read_pickle("/content/drive/MyDrive/Hackathon_HiGency/data/pred_carbon_df.pkl")

In [31]:
carbon_pred_np.sum()

86819.7

In [32]:
# Verification
df_walid[['x_max','x_min','y_max','y_min']].values.sum()

822054

In [33]:
# Add the e
carbon_pred_df['e'] = carbon_pred_np

In [34]:
# Drop the class class
carbon_pred_df = carbon_pred_df.drop(['class'], axis=1)

In [35]:
# Append the non-cars
image_car = set(carbon_pred_df['im_name'])
for imagepath in glob.glob("/content/drive/MyDrive/Hackathon_HiGency/data/dataset_set/*.jpg"):
  if not imagepath.split('/')[-1] in image_car:
    entry = {
        'im_name' : imagepath.split('/')[-1],
        'x_min' : 0.0,
        'y_min' : 0.0,
        'x_max' : 0.0,
        'y_max' : 0.0,
        'e'     : 0.0,
    }
    carbon_pred_df = carbon_pred_df.append(entry, ignore_index=True)
carbon_pred_df = carbon_pred_df.reset_index()
carbon_pred_df = carbon_pred_df.drop(['index'], axis=1)

In [54]:
carbon_pred_df.to_csv('group7.csv')

In [55]:
carbon_pred_df

,im_name,x_min,y_min,x_max,y_max,e
0,71540198.jpg,98,79,1620,815,204.455017
1,85427036.jpg,234,195,1033,643,152.784576
2,6087324064.jpg,1480,371,1602,705,86.715759
3,11828073.jpg,19,62,614,417,141.095856
4,5116251.jpg,96,77,1081,559,145.025482
...,...,...,...,...,...,...
1018,6858047610.jpg,0.0,0.0,0.0,0.0,0.000000
1019,16398172267.jpg,0.0,0.0,0.0,0.0,0.000000
1020,9797073.jpg,0.0,0.0,0.0,0.0,0.000000
1021,6189237.jpg,0.0,0.0,0.0,0.0,0.000000


In [51]:
# from PIL import Image
# import matplotlib.pyplot as plt
# def display_box(df):
#     for i in range(df.shape[0]):
        
#         im_test = df.iloc[i, 0]
#         image = np.asanyarray(Image.open(f"/content/drive/MyDrive/Hackathon_HiGency/data/dataset_set/{im_test}"))
        
#         cv2.rectangle(image, (df.iloc[i, 1], df.iloc[i, 2]), (df.iloc[i, 3], df.iloc[i, 4]), (255,0,0), 2)
        
#         print(im_test)
#         print("e = ", df.iloc[i, -1])
#         plt.imshow(image)
#         plt.show()

In [ ]:
# display_box(carbon_pred_df)

In [37]:
!mkdir -p tmp

In [42]:
# Draw images
import cv2
from tqdm import tqdm

data_path = '/content/drive/MyDrive/Hackathon_HiGency/data/dataset_set/'
walker = glob.glob(data_path + "*.jpg")

for filepath in tqdm(walker):
  filename = filepath.split('/')[-1]
  img = cv2.imread(data_path + filename)
  entry = carbon_pred_df[carbon_pred_df['im_name'] == filename].iloc[0]
  x_min, y_min, x_max, y_max = entry[['x_min', 'y_min', 'x_max', 'y_max']]
  e = entry['e']
  if e > 0:
    img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (255,0,0), 2)
  else:
    img = cv2.circle(img, (int(img.shape[0] / 2), int(img.shape[1] / 2)), 30, (255,0,0), 2)
  cv2.imwrite("tmp/{}".format('e_' + str(int(entry['e'])) + '_' + filename), img)

100%|██████████| 1023/1023 [01:31<00:00, 11.21it/s]


In [43]:
!zip -r our_images.zip tmp/

  adding: tmp/ (stored 0%)
  adding: tmp/e_0_4505535858.jpg (deflated 6%)
  adding: tmp/e_172_7206587.jpg (deflated 10%)
  adding: tmp/e_172_40472278.jpg (deflated 10%)
  adding: tmp/e_172_40822312.jpg (deflated 6%)
  adding: tmp/e_0_2428834052.jpg (deflated 4%)
  adding: tmp/e_159_99289575.jpg (deflated 6%)
  adding: tmp/e_150_79321410.jpg (deflated 9%)
  adding: tmp/e_179_64251477.jpg (deflated 10%)
  adding: tmp/e_148_75483353.jpg (deflated 8%)
  adding: tmp/e_148_76188745.jpg (deflated 8%)
  adding: tmp/e_0_2864688.jpg (deflated 5%)
  adding: tmp/e_0_4923015829.jpg (deflated 4%)
  adding: tmp/e_0_7961266004.jpg (deflated 10%)
  adding: tmp/e_182_70745147.jpg (deflated 8%)
  adding: tmp/e_153_41276182.jpg (deflated 11%)
  adding: tmp/e_175_21608766.jpg (deflated 6%)
  adding: tmp/e_192_6212736114.jpg (deflated 7%)
  adding: tmp/e_0_5971556958.jpg (deflated 13%)
  adding: tmp/e_249_25603258.jpg (deflated 10%)
  adding: tmp/e_181_983702.jpg (deflated 9%)
  adding: tmp/e_0_9274515162.j

In [44]:
!cp our_images.zip /content/drive/MyDrive/Hackathon_HiGency/data/